In [ ]:
/* create the first purchase date and calculate the month order - # of months after first purchase */

SELECT y.first_purchase, y.month_order,y.Customers, y.Nr_Orders, y.SalesEx, y.Units,
       y.CohortCustomers, y.repurchase_rate,y.Nr_Orders/y.Customers AS FOS, ROUND(y.Units/y.Nr_Orders,1) AS AOU, 
       y.SalesEx/y.Units AS AISP,ROUND(y.SalesEx/y.Nr_Orders,0) AS AOV
    
FROM(
WITH t_first_purchase AS (
  SELECT 
  date,
  DATE_DIFF(date, first_purchase_date, MONTH) AS month_order,
  FORMAT_DATETIME('%Y%m', first_purchase_date) AS first_purchase,
  customer_id,
  order_item_sales,
  order_id,
  Units,
  FROM (
    SELECT 
    DATE(TIMESTAMP(Auth_Date)) AS date,
    customer_id,
    order_item_sales,
    order_id,
    Units,
    FIRST_VALUE(DATE(TIMESTAMP(Auth_Date))) OVER (PARTITION BY customer_id ORDER BY DATE(TIMESTAMP(Auth_Date))) AS first_purchase_date
    FROM 
    (
    SELECT *,
CASE WHEN customer_seq > 1 THEN 'Repeat Customer'
ELSE 'New Customer'
END AS RepeatPurchase
FROM (
    SELECT  *,
    RANK() OVER (PARTITION BY customer_id ORDER BY x.Auth_Date) AS customer_seq
    FROM
(
SELECT  
       DISTINCT customer_id,
       DATE(o.order_authed_timestamp) AS Auth_Date,
        o.order_id,
        oi.order_item_sales,
        oi.order_item_quantity AS Units
    FROM `tal-production-data-bi.tal_dm_order.orders_2010_2020` o,
        UNNEST(o.order_items) oi
    WHERE   
        DATE(o.order_placed_timestamp) >= "2019-04-01" AND DATE(o.order_placed_timestamp) < "2020-01-01" 
        AND DATE(o.order_authed_timestamp) >= "2019-04-01" AND DATE(o.order_authed_timestamp) < "2020-01-01" 
        AND o.order_authorisation_status IN ('Auth')
        AND o.is_not_return_cancel_in_full IS TRUE
        AND o.voucher_sales_percent =0
        AND is_test_customer = false
        AND o.order_authed_timestamp IS NOT NULL

UNION ALL

SELECT 
        DISTINCT customer_id,
        DATE(o.order_authed_timestamp) AS Auth_Date,
        o.order_id,
        oi.order_item_sales,
        oi.order_item_quantity AS Units
    FROM `tal-production-data-bi.tal_dm_order.orders_2020_2030` o,
        UNNEST(o.order_items) oi
    WHERE   
        DATE(o.order_placed_timestamp) >= "2020-01-01" AND DATE(o.order_placed_timestamp) < "2022-05-01" 
        AND DATE(o.order_authed_timestamp) >= "2020-01-01" AND DATE(o.order_authed_timestamp) < "2022-05-01" 
        AND o.order_authorisation_status IN ('Auth')
        AND o.is_not_return_cancel_in_full IS TRUE
        AND o.voucher_sales_percent =0
        AND is_test_customer = false
        AND o.order_authed_timestamp IS NOT NULL) x)  
    )
    )
  ),

/* aggregate customer count per first purchase cohort and month order */
t_agg AS (
  SELECT 
  first_purchase,
  month_order,
  COUNT(DISTINCT customer_id) AS Customers,
  COUNT(DISTINCT order_id) AS Nr_Orders,
  SUM(order_item_sales) AS SalesEx,
  SUM(Units) AS Units
  FROM 
  t_first_purchase
  GROUP BY first_purchase, month_order
),

/* repurchase rate */
 t_cohort AS (
  SELECT *,
  SAFE_DIVIDE(Customers, CohortCustomers) AS repurchase_rate
  FROM (
      SELECT *,
      FIRST_VALUE(Customers) OVER (PARTITION BY first_purchase ORDER BY month_order) AS CohortCustomers
      FROM t_agg
  )
 )
SELECT * FROM t_cohort 
ORDER BY first_purchase, month_order) y
GROUP BY 1,2,3,4,5,6,7,8